In [1]:
'''
reading all libraries and packages that we need
'''
import numpy as np
from matplotlib import pyplot as plt
import os
import sys
from skimage import io
import pandas as pd
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from skimage.segmentation import mark_boundaries
from skimage.measure import regionprops, label
import SimpleITK as sitk
from scipy.ndimage import binary_fill_holes
from scipy import ndimage as ndi
from skimage.morphology import closing, square, remove_small_objects, binary_erosion, disk, binary_dilation
from skimage.segmentation import clear_border
from skimage.color import label2rgb
import time

from skimage.filters import  threshold_otsu, threshold_triangle, gaussian, threshold_local
from skimage.morphology import convex_hull_image
from skimage.draw import line, polygon
import plotly
from plotly.offline import plot 
from scipy.stats import kstest
from scipy.stats import ks_2samp
import pickle
from collections import OrderedDict
from collections import Counter
from scipy.stats import kstest
from scipy.stats import ks_2samp
import mrc
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

In [3]:
def prune_and_label_dapi(seg_dapi, debris_size, erosion_radius):
    ''' Segments the dapi image.
    Ipnuts:
    
    seg_dapi: 2-dimensional numpy array (segmented image array)
    debris_size: size of the debris to be removed (in pixels)
    erosion_radius: radius (in pixels) of the disk to be used for binary ersion to separate connected nuclei
    
    Outputs:
    nuclear_mask: segmented nuclei image
    nuclear_labels: labled nuclei
    
    '''
    # remove small debris from the segmented imaage
    seg_dapi_1 = remove_small_objects(seg_dapi, debris_size)
    
    seg_dapi_2 = binary_erosion(seg_dapi_1, disk(2))
    
    # fill the holes in the image
    nuclear_mask = ndi.binary_fill_holes(seg_dapi_2)

    
#     nuclear_mask = remove_small_objects(nuclear_mask, min_area_to_keep_cell)
    
    # erode the image
    eroded_mask = binary_erosion(nuclear_mask, disk(erosion_radius))

    distance = ndi.distance_transform_edt(eroded_mask)
    local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((1, 1)),
                                labels=nuclear_mask)
    markers = ndi.label(local_maxi)[0]
    nuclear_labels = watershed(-distance, markers, mask=nuclear_mask, connectivity=2)
    
    return nuclear_mask, nuclear_labels

In [4]:
def segment_and_label_dapi(dapi_image, debris_size, min_area_to_keep_cell, erosion_radius, block_size = 251):
    ''' Segments the dapi image.
    Ipnuts:
    
    dapi_image: 2-dimensional numpy array (image array)
    debris_size: size of the debris to be removed (in pixels)
    erosion_radius: radius (in pixels) of the disk to be used for binary ersion to separate connected nuclei
    
    Outputs:
    nuclear_mask: segmented nuclei image
    nuclear_labels: labled nuclei
    
    '''
#     temp_seg_1 = dapi_image > threshold_otsu(dapi_image)
    # locad thresholding to segment the nuclei
    adaptive_thresh = threshold_local(dapi_image, block_size = block_size)
    seg_dapi = dapi_image > adaptive_thresh
    
#     seg_dapi = temp_seg_1*temp_seg_2
    
    # use Otsu method to segment the dapi image
#     seg_dapi = dapi_image > threshold_otsu(dapi_image)

    # fill the holes in the image
    nuclear_mask = ndi.binary_fill_holes(seg_dapi)
    
    # remove small debris from the segmented imaage
    seg_dapi = remove_small_objects(seg_dapi, debris_size)

    
    
    nuclear_mask = remove_small_objects(nuclear_mask, min_area_to_keep_cell)
    
    # erode the image
    eroded_mask = binary_erosion(nuclear_mask, disk(erosion_radius))

    distance = ndi.distance_transform_edt(eroded_mask)
    local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((1, 1)),
                                labels=nuclear_mask)
    markers = ndi.label(local_maxi)[0]
    nuclear_labels = watershed(-distance, markers, mask=nuclear_mask, connectivity=2)
    
    return nuclear_mask, nuclear_labels

In [5]:
def obtain_big_cells(labeled_img, area_thresh, cir_thresh):
        ''' Determining the package of the cells as a big cell.
    Ipnuts:
    
    labeled_img: nuclei label from the labling function
    area_thresh: maximum size of a nucleus (in pixels) to be considered as a single nucleus
    cir_thresh: circularity threshold to see whether it is a single nucleus or multiple nuclei
    
    Outputs:
    big_cells: label of the cells that are satistfying the area AND circularity threshold
    
    '''
    circ = lambda r: (4 * np.pi * r.area) / (r.perimeter * r.perimeter)
    big_cells = [(prop.label, prop.area, circ(prop)) for prop in regionprops(labeled_img)
                 if prop.area > area_thresh and circ(prop) < cir_thresh]
    return big_cells

In [6]:
def split_cells(label_rem, image):
            ''' Splitting a package of the cells
    Ipnuts:
    
    labeled_rem: label of the object that should be splitted
    image: image: 2D labeled image
    
    Outputs:
    labels_rem: label of the cells that are going to split
    
    '''
    
    relabels, tmp_img = np.zeros_like(image), np.zeros_like(image)
    pts = []
    row_cords =[]
    col_cords = []
    for prop in regionprops(label_rem):
        x,y = np.int16(np.round(prop.centroid))
        row_cords.append(x)
        col_cords.append(y)
        pts.extend([x,y])
    if len(regionprops(label_rem))>2:
        rr, cc = polygon(row_cords, col_cords)
    else:
        rr, cc = line(pts[0], pts[1], pts[2], pts[3])

    tmp_img[rr,cc] =1
    tmp_img = binary_dilation(tmp_img, disk(10))
    tmp_img = convex_hull_image(tmp_img)
    split_img = np.logical_and(image, np.logical_not(tmp_img))
    distance_rem = ndi.distance_transform_edt(split_img)
    local_maxi_rem = peak_local_max(distance_rem, indices=False, footprint=np.ones((1, 1)),
                                labels=image)
    markers_rem = label(local_maxi_rem, connectivity=2)
    labels_rem = watershed(-distance_rem, markers_rem, mask=image, connectivity=2)
    return labels_rem

In [7]:
def obtain_final_labels_after_splitting_big_objects(nuclear_labels, big_cells, 
                                                    ero_rad, min_rem_area, min_area_to_keep_cell):
    '''
    Inputs:
    nuclear_labels: 2D numpy labeled dapi image
    big_cells: label of the big objects
    ero_rad: radius of the disk to be used for eroading the region (convex hull - roi)
    min_rem_area: removing any object less than of this size (in pixels)
    min_area_to_keep_cell: minimum size of an acceptable nucleus (in mpixels)
    
    Outputs:
    final_label: final label of nuecleus after removing small objects and splitting big packages
    '''
    
    relabels = np.zeros_like(nuclear_labels)

    for ii, area, cir in big_cells:
        image = nuclear_labels==ii
        chull = convex_hull_image(image)
        rem = np.logical_and(chull, np.logical_not(image))
        eroded_rem = binary_erosion(rem, disk(ero_rad))
        eroded_rem = remove_small_objects(eroded_rem, min_rem_area)
        label_rem = label(eroded_rem)

        if np.max(label_rem) >= 2:
            relabels = label(relabels + split_cells(label_rem, image))## split cells

    assigned_relabels = np.zeros_like(relabels)
    for p in regionprops(relabels):
        if p.label > 0:
            assigned_relabels[np.where(relabels==p.label)] = np.max(nuclear_labels) + p.label

    label_img = label(nuclear_labels + assigned_relabels)
    label_img = remove_small_objects(label_img, min_area_to_keep_cell)
#     final_labels = clear_border(label_img)
    final_labels = label_img
    return final_labels

In [9]:
def create_dir(dir_name):
    
    if not os.path.exists(dir_name):
            os.makedirs(dir_name)
    return None

In [12]:
root_path = r"D:\12-January21\GREB1\FVWAE2"

In [13]:
fpaths = [os.path.join(root_path,f) for f in os.listdir(root_path) if f.endswith('.dv')]

In [14]:
# Default Values
sigma = 1
erosion_radius = 31
debris_size = 100
min_area_to_keep_cell = 3000
block_size = 651
dilation_radius = 100
area_thresh =20000# maximum area of the cell to be considered one cell
cir_thresh = 0.70
ero_rad = 5 # 5 for DV, radius of the disk to be used for eroading the region (convex hull - roi)
min_rem_area = 2 # 2 for DV

In [15]:
seg_dir = os.path.join(root_path, 'Segmentation_erosion_radius_'+ str(erosion_radius)
                       + '_min_area_' + str(min_area_to_keep_cell) +'_area_thresh_' + str(area_thresh))

if not os.path.exists(seg_dir):
    os.makedirs(seg_dir)

In [16]:
#creating folders for the processed images
raw_img_dir = os.path.join(os.path.dirname(seg_dir), 'raw')
raw_img_dir_leave_top_bottom = os.path.join(os.path.dirname(seg_dir), 'raw_projected')
seg_dapi_dir = os.path.join(os.path.dirname(seg_dir), 'seg_dapi')
for dir_name in [raw_img_dir, raw_img_dir_leave_top_bottom, seg_dapi_dir]:
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)


In [18]:
features = pd.DataFrame()
for fpath in fpaths:
    fname = os.path.basename(fpath)
    main_fname = ('_').join(fname.split('_')[0:])
    if 'wash' in fname:
        fname_noext = os.path.splitext(fname)[0]
        year, mon_date,  gene,_,_,_,time,field, _, _ =fname_noext.split('_')
        # Store features
        features = features.append([{'filename': main_fname,
                                         'filepath': fpath,
                                         'date': ('').join([year,mon_date]),
                                         'gene': gene,
                                         'time': time,
                                         'field' : field,
                                          },])
    else:
        fname_noext = os.path.splitext(fname)[0]
        year, mon_date, gene,_,field,_,_ =fname_noext.split('_')
        # Store features
        features = features.append([{'filename': main_fname,
                                         'filepath': fpath,
                                         'date': ('').join([year,mon_date]),
                                         'gene': gene,
                                         'time': '0',
                                         'field' : field,
                                          },])

features = features.reset_index(drop=True)

In [22]:
# Max projecting the images
starting_time = time()

for fpath in features['filepath'].unique():
    st_time = time()
    fname = os.path.splitext(os.path.basename(fpath))[0] + '.tif'
    
    df = features[features['filepath'] == fpath]
    t=df.iloc[0]['time']
    f=df.iloc[0]['field']
#     print(df['filepath'].values[0])
    img_array = mrc.imread(fpath)
    DIM=img_array.shape[1]
#     print(img_array.shape)

    #############################
    dapi_image = np.max(img_array[0], axis=0)
    io.imsave(os.path.join(raw_img_dir, 'dapi_' + fname), dapi_image)
    
    dapi_image = np.max(img_array[0, 2:DIM-1], axis=0)
    io.imsave(os.path.join(raw_img_dir_leave_top_bottom, 'dapi_' + fname), dapi_image)
    
    exon_image = np.max(img_array[1], axis=0)
    io.imsave(os.path.join(raw_img_dir,'exon_' + fname), exon_image)
    
    exon_image = np.max(img_array[1, 2:DIM-1], axis=0)
    io.imsave(os.path.join(raw_img_dir_leave_top_bottom,'exon_' + fname), exon_image)
    
    intron_image = np.max(img_array[2], axis=0)
    io.imsave(os.path.join(raw_img_dir,'intron_' + fname), intron_image)
    
    intron_image = np.max(img_array[2, 2:DIM-1], axis=0)
    io.imsave(os.path.join(raw_img_dir_leave_top_bottom,'intron_' + fname), intron_image)
#     dapi_image =  io.imread(os.path.join(fpath, fname))
    # get the nuclear mask and nuclear labels
    nuclear_mask, nuclear_labels = segment_and_label_dapi(dapi_image, debris_size, 
                                                          min_area_to_keep_cell,
                                                          erosion_radius,
                                                         block_size = block_size)

    # get the big cells (to be split if it contains more than 1 cells)
    big_cells = obtain_big_cells(nuclear_labels, area_thresh, cir_thresh)

    # get the final labels after splitting the big cells
    final_labels = obtain_final_labels_after_splitting_big_objects(nuclear_labels, big_cells, ero_rad, 
                                                min_rem_area, min_area_to_keep_cell)
    # nuclei not touching border
    non_border_labels = clear_border(final_labels)
    nuc_mask = final_labels > 0
    marked_dapi = mark_boundaries(dapi_image, non_border_labels, color=(1, 1, 1), outline_color=(1, 1, 1))
    io.imsave(os.path.join(seg_dapi_dir, 'dapi_' + fname), np.uint8(marked_dapi*255))
print(time() - starting_time )

In [21]:
newfpaths = [os.path.join(raw_img_dir_leave_top_bottom,f) for f in os.listdir(raw_img_dir_leave_top_bottom) if f.endswith('.tif')]

In [36]:
#Segmenting DAPI channel
alpha = 40
st_time = time()

for fpath in newfpaths:
    if 'dapi' in fpath:
        
    #     print(fpath)
        dapi_image = io.imread(fpath)
        f = dapi_image
        
        blurred_f = ndi.gaussian_filter(f, 3)

        filter_blurred_f = ndi.gaussian_filter(blurred_f, 1)

        
        sharpened = blurred_f + alpha * (blurred_f - filter_blurred_f)

        filled_seg_dapi = ndi.binary_fill_holes(dapi_image > (0.6*threshold_otsu(dapi_image)))

        adaptive_thresh = threshold_local(sharpened, block_size = block_size)

        temp_seg_2 = ndi.binary_fill_holes (sharpened > adaptive_thresh)
        
        seg_dapi = np.logical_and(filled_seg_dapi, temp_seg_2)

        fname = os.path.basename(fpath)

        io.imsave(os.path.join(seg_dapi_dir, 'seg_' + fname), np.uint8(255*seg_dapi))
#         io.imsave(os.path.join(seg_dapi_dir, 'seg_2_' + fname), np.uint8(255*temp_seg_2))
        
print('Time taken', time()- st_time, 'seconds.')

Time taken 366.18459820747375 seconds.


In [23]:
newfeatures = pd.DataFrame()
for fpath in newfpaths:
    fname = os.path.basename(fpath)
    main_fname = ('_').join(fname.split('_')[1:])
    if 'wash' in fname:
        fname_noext = os.path.splitext(fname)[0]
        wavelength, year, mon_date, _,_,_,_,time,field, _, _ =fname_noext.split('_')
        # Store features
        newfeatures = newfeatures.append([{'filename': main_fname,
                                         'filepath': fpath,
                                         'date': ('').join([year,mon_date]),
                                         'time': time,
                                         'field' : field,
                                     'wavelength' : wavelength,
                                          },])
    else:
        fname_noext = os.path.splitext(fname)[0]
        wavelength, year, mon_date, _,_, field, _, _ =fname_noext.split('_')
        # Store features
        newfeatures = newfeatures.append([{'filename': main_fname,
                                         'filepath': fpath,
                                         'date': ('').join([year,mon_date]),
                                         'time': '0',
                                         'field' : field,
                                     'wavelength' : wavelength,
                                          },])

newfeatures = newfeatures.reset_index(drop=True)
newfeatures = newfeatures.astype({"field": int})


In [25]:
t = {'15min': 15, '30min':30, '45min': 45, '60min': 60, '75min': 75, '90min':90,'0':0}
newfeatures["time"] = newfeatures["time"].map(t)

In [26]:
seg_dir = os.path.join(root_path,  'Segmentation_erosion_radius_'+ str(erosion_radius)
                       + '_min_area_' + str(min_area_to_keep_cell) +'_area_thresh_' + str(area_thresh))

if not os.path.exists(seg_dir):
    os.makedirs(seg_dir)

In [34]:
def distance(nuc,intr):
    return np.sqrt(np.square(nuc[0]-intr[0])+np.square(nuc[1]-intr[1]))

In [35]:
#Using Dictionary for saving the results
def create_dicts():
    
    summary = {}

    Mature={}
    Mature['inside']={}
    Mature['inside_intensities']={}
    Mature['outside_mean_intensities']={}
    Mature['outside_integ_intensities']={}
    Mature['outside_integ_area']={}
    Mature['outside']={}
    Mature['outside_area']={}
    Mature['coloc']={}
    threshold={}
    
    return summary, Mature




In [37]:
#Final 
fpath = raw_img_dir_leave_top_bottom
threshold={}
threshold['exon']={}
threshold['intron']={}
temp_dapis = {}
temp_introns = {}
threshold_df = pd.DataFrame()

for ii, room in enumerate(elbow_rooms):
    threshold['exon'][room]={}
    threshold['intron'][room]={}
    starting_time = time()
    
    append_name = '_' + str(np.abs(room))
    
    # create dictionaries
    summary, Mature = create_dicts()
    for file_name in newfeatures['filename'].unique():
#         print(file_name)
#         t_time = time()
        df = newfeatures[newfeatures['filename'] == file_name]
        t=df.iloc[0]['time']#str(df.iloc[0]['time']) + "_" + df.iloc[0]['treatment']
        f=df.iloc[0]['field']
#         threshold['exon'][room]['time' +str(t)+'field'+str(f)]=[]
#         threshold['intron'][room]['time' +str(t)+'field'+str(f)]=[]
        for channel in df['wavelength']:
            sdf = df[df['wavelength']==channel]
    #         fname = sdf['filename'].get_values()[0]
            fname = os.path.basename(list(sdf['filepath'])[0])

            if channel == 'dapi':
                if ii == 0:
                    seg_name = 'seg_' + fname
                    seg_dapi =  io.imread(os.path.join(seg_dapi_dir, seg_name))
                    # get the nuclear mask and nuclear labels
                    nuclear_mask, nuclear_labels = prune_and_label_dapi(seg_dapi, debris_size, erosion_radius)


                    # get the big cells (to be split if it contains more than 1 cells)
                    big_cells = obtain_big_cells(nuclear_labels, area_thresh, cir_thresh)

                    # get the final labels after splitting the big cells
                    final_labels_1 = obtain_final_labels_after_splitting_big_objects(nuclear_labels, big_cells, ero_rad, 
                                                                min_rem_area, min_area_to_keep_cell)

                    nuc_mask = final_labels_1

                    # dialte the nuclear mask to get back to right size
                    temp_img = np.logical_or(binary_dilation(nuc_mask, disk(2)), binary_fill_holes(seg_dapi))

                    # use watershed method with final_labels as markers to obtain labeled cell mask
                    final_labels = watershed(temp_img, markers=final_labels_1, mask=temp_img)

                    temp_dapis[fname] = (seg_dapi, final_labels)

                    # nuclei not touching border
                    non_border_labels = clear_border(final_labels)
                    dil_seg_dapi = binary_dilation(nuc_mask, disk(100))
                    seg_cyto = np.logical_xor (dil_seg_dapi, nuc_mask)
                    nucloc=np.asarray(np.where((nuc_mask==1))).T
                    noncyt=np.asarray(np.where((seg_cyto==0))).T
                    
                else:
                    seg_dapi, final_labels = temp_dapis[fname]
                    
                    # nuclei not touching border
                    non_border_labels = clear_border(final_labels)



            elif channel == 'exon':
                exon_image =  io.imread(os.path.join(fpath, fname))
    #             exon_image = exon_image / np.median(exon_image)
                seg_exon = exon_image > (threshold_otsu(exon_image))*(1 + room/100)
                threshold['exon'][room]['time' +str(t)+'field'+str(f)]=(threshold_otsu(exon_image))*(1 + room/100)
                
            elif channel == 'intron':
                
                if ii == 0:
                    intron_image = sitk.ReadImage(os.path.join(fpath, fname))
                    gaussian_blur = sitk.SmoothingRecursiveGaussianImageFilter()
                    gaussian_blur.SetSigma ( float ( sigma ) )
                    blur_intron = gaussian_blur.Execute ( intron_image )

                    max_entropy_filter = sitk.MaximumEntropyThresholdImageFilter()
                    max_entropy_filter.SetInsideValue(0)
                    max_entropy_filter.SetOutsideValue(1)
                    seg = max_entropy_filter.Execute(blur_intron)
                    seg_intron = sitk.GetArrayFromImage(seg)

                    blur_intron_img = sitk.GetArrayFromImage(blur_intron)

                    intron_threshold = np.min(blur_intron_img[np.where(seg_intron != 0)])

                    temp_introns[fname] = (blur_intron_img, intron_threshold)

                    seg_intron = blur_intron_img > intron_threshold
                    threshold['intron'][room]['time' +str(t)+'field'+str(f)]=intron_threshold
                    
                else:
                    blur_intron_img, intron_threshold = temp_introns[fname]
                    seg_intron = blur_intron_img > (intron_threshold*(1 + room/100))
                    threshold['intron'][room]['time' +str(t)+'field'+str(f)]=(intron_threshold*(1 + room/100))


            else:
                print('Different than dapi, exon or intron image found!')
                
        if room ==0:
            comb_img = np.zeros((seg_exon.shape[0], seg_exon.shape[0],3))
            comb_img[:,:,0] = seg_intron
            comb_img[:,:,1] = seg_exon
            comb_img[:,:,2] = nuc_mask
            marked_img = mark_boundaries(comb_img, non_border_labels, color=(1, 1, 1), outline_color=(1, 1, 1))
            io.imsave(os.path.join(seg_dir, 'combined_' + fname), np.uint8(marked_img*255))
            
        Mature['inside']['time' +str(t)+'field'+str(f)]=np.array([])
        Mature['outside']['time' +str(t)+'field'+str(f)]=np.array([])
        Mature['outside_area']['time' +str(t)+'field'+str(f)]=np.array([])
        Mature['outside_mean_intensities']['time' +str(t)+'field'+str(f)]=np.array([])
        Mature['outside_integ_intensities']['time' +str(t)+'field'+str(f)]=np.array([])
        Mature['outside_integ_area']['time' +str(t)+'field'+str(f)]=np.array([])
        Mature['inside_intensities']['time' +str(t)+'field'+str(f)]=np.array([])
        Mature['coloc']['time' +str(t)+'field'+str(f)]=np.array([])

        green=label(seg_exon)
        red=label(seg_intron)
        blue=label(non_border_labels)
        
        green2=label(seg_exon2)
        
        for exon2 in regionprops(green2,intensity_image=exon_image2):
            Mature['outside']['time' +str(t)+'field'+str(f)]=np.append(Mature['outside']['time' +str(t)+'field'+str(f)],exon2.label)
            Mature['outside_area']['time' +str(t)+'field'+str(f)]=np.append(Mature['outside_area']['time' +str(t)+'field'+str(f)],exon2.area)
            Mature['outside_mean_intensities']['time' +str(t)+'field'+str(f)]=np.append(Mature['outside_mean_intensities']['time' +str(t)+'field'+str(f)],exon2.mean_intensity)
            Mature['outside_integ_intensities']['time' +str(t)+'field'+str(f)]=np.append(Mature['outside_integ_intensities']['time' +str(t)+'field'+str(f)],sum(sum(exon2.intensity_image)))
            Mature['outside_integ_area']['time' +str(t)+'field'+str(f)]=np.append(Mature['outside_integ_area']['time' +str(t)+'field'+str(f)],\
                                                                              [sum(sum(exon2.intensity_image)),exon2.area])
    

#         print(time()-t_time, 'seconds')
        
        # threshold values for different files with different elbow room allowed
        threshold_df = threshold_df.append([{'filename': file_name,
                                         'elbow_room': room,
                                         'exon_thresh': (threshold_otsu(exon_image))*(1 + room/100),
                                         'intron_thresh': (intron_threshold*(1 + room/100)),
                                         'time': t,
                                         'field' : f,
                                     'rough_estimate_num_exons': np.max(green),
                                     'rough_estimate_num_introns': np.max(red),
                                          },])



        summary['time' +str(t)+'field'+str(f)]={}

        ### get measurements
        # number of cells not toching border

        for nuc in regionprops(blue):
            nuc_id=nuc.label
            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]={}
            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['intron']=[]

            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['exon']=[]
            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['nasexon']=[]

            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['colocspot']=[]
            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['colocspotarea']=[]
            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['colocspotarea2']=[]
            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['colocspotmi']=[]
            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['colocspotcenter']=[]
            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['distarea']=[]
            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['Ncolocspot']=[]
            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['coord_nas']=[]
            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['nascent']=[]
            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['Nnascent']=[]

            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['maturein']=[]
            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['integ_intens']=[]

            for exon in regionprops(green,intensity_image=exon_image):
                exon_id=exon.label
                if exon.area<200:
                    if (((exon.coords[:, None] == nuc.coords).all(-1).any(-1)==True).any()):
                        summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['exon'].append(exon_id)

            for intron in regionprops(red,intensity_image=sitk.GetArrayFromImage(intron_image)):
                intron_id=intron.label
                if (((intron.coords[:, None] == nuc.coords).all(-1).any(-1)==True).all()):
                    summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['intron'].append(intron_id)         

                    for nucexon in summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['exon']:
                        if (((intron.coords[:, None] == regionprops(green)[nucexon-1].coords).all(-1).any(-1)==True).any()):
                            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['colocspot'].append([intron_id,nucexon])
                            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['colocspotarea2'].append([regionprops(red)[intron_id-1].area,regionprops(green)[nucexon-1].area])
                            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['colocspotmi'].append([regionprops(red,intensity_image=\
                                                                                                                     sitk.GetArrayFromImage(intron_image))[intron_id-1].mean_intensity,regionprops(green,intensity_image=exon_image)[nucexon-1].mean_intensity])
                            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['distarea'].append([distance(nuc.centroid,regionprops(red)[intron_id-1].centroid),regionprops(red)[intron_id-1].area])
                            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['integ_intens'].append(sum(sum(regionprops(green,intensity_image=exon_image)[nucexon-1].intensity_image)))
                            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['nasexon'].append(np.sum(regionprops(green,intensity_image=exon_image)[nucexon-1].intensity_image))
                        
                            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['coord_nas'].append(regionprops(green)[nucexon-1].coords)
                            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['colocspotcenter'].append([regionprops(red)[intron_id-1].centroid,regionprops(green)[nucexon-1].centroid])
                            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['colocspotarea'].append(np.count_nonzero(((regionprops(red)[intron_id-1].coords[:, None]==regionprops(green)[nucexon-1].coords).all(-1).any(-1))==True))
                            if ((np.count_nonzero(((regionprops(red)[intron_id-1].coords[:, None]==regionprops(green)[nucexon-1].coords).all(-1).any(-1))==True))<2):
                                summary_warning['onepixel'].append([t,f,nuc_id,regionprops(red)[intron_id-1].centroid,regionprops(green)[nucexon-1].centroid])
   

            X=np.array(summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['colocspot'])
            W= np.array(summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['intron'])
            Y=np.array(summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['exon'])

            if(len(X)>0):
                summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['maturein']=np.setdiff1d(Y,X[:,1])
                if((len(X[:,0])>len(np.unique(X[:,0]))) or(len(X[:,1])>len(np.unique(X[:,1])))) :
                    unique, counts = np.unique(X[:,0], return_counts=True)
                    unique2, counts2 = np.unique(X[:,1], return_counts=True)
                    if((len(X[:,0])>len(np.unique(X[:,0])))):
                        for i in (unique[(counts)>1]):
                            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['nascent'].append([i,X[X[:,0]==i][:,1]])

                        v=([X[X[:,0]==i][0][0] for i in unique[(counts)==1]])
                        u=([X[X[:,0]==i][0][1] for i in unique[(counts)==1]])
                        r=([X[X[:,0]==i][0] for i in unique[(counts)==1]])
                        if (len(u)>len(np.unique(u))):
                            for j in np.unique(u):
                                summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['nascent'].append([X[X[:,1]==j][:,0],j])
                        elif(len(r)>0):
                            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['nascent']=np.vstack((summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['nascent'],r))

                    else:
                        for k in (unique2):

                            summary_warning['Warningex'].append([t,f,nuc_id,regionprops(green)[k-1].centroid])
                            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['nascent'].append([X[X[:,1]==k][:,0],k]) 
                else:
                    summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['nascent']=summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['colocspot']
            else:
                summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['maturein']=summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['exon']

            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['Ncolocspot']=len(summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['colocspot'])
            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['Nnascent']=len(summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['nascent'])

   
        print('t = ' ,t,'min, f = ',f,", ", time()-starting_time, 'seconds')
        
    with open(os.path.join(seg_dir, 'summaryJan2FVWAE2' + append_name + '.pickle'), 'wb') as handle:
        pickle.dump(summary, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open(os.path.join(seg_dir, 'MatureJan2FVWAE2' + append_name + '.pickle'), 'wb') as handle:
        pickle.dump(Mature, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    print('Elbow room (in %) = ' , room, ', total time taken :',  time()-starting_time, 'seconds')
        
threshold_df = threshold_df.reset_index(drop=True)

with open(os.path.join(seg_dir, 'thresholdFVWAE2.pickle'), 'wb') as handle:
        pickle.dump(threshold, handle, protocol=pickle.HIGHEST_PROTOCOL)

t =  0 min, f =  1 ,  4865.088057041168 seconds
t =  0 min, f =  2 ,  9572.126098632812 seconds
t =  0 min, f =  3 ,  16102.116491556168 seconds
t =  0 min, f =  4 ,  22191.698852300644 seconds
t =  0 min, f =  5 ,  26005.915694236755 seconds
t =  15 min, f =  1 ,  30011.216656684875 seconds
t =  15 min, f =  2 ,  34432.01913714409 seconds
t =  15 min, f =  3 ,  38288.918508291245 seconds
t =  15 min, f =  4 ,  41829.76835465431 seconds
t =  15 min, f =  5 ,  45325.14707159996 seconds
t =  15 min, f =  6 ,  48965.3746650219 seconds
t =  15 min, f =  7 ,  53057.42999911308 seconds
t =  15 min, f =  8 ,  57659.48322844505 seconds
t =  15 min, f =  9 ,  61922.98404383659 seconds
t =  15 min, f =  10 ,  68758.90012741089 seconds
t =  30 min, f =  1 ,  74562.23758292198 seconds
t =  30 min, f =  2 ,  79176.949821949 seconds
t =  30 min, f =  3 ,  83552.5816833973 seconds
t =  30 min, f =  4 ,  87700.20801973343 seconds
t =  30 min, f =  5 ,  92187.76592946053 seconds
t =  30 min, f =  6 ,  